# Directed Graph Utils

Utility functions to calculate the magnetic laplacian and perform other common tasks in directed graphs

In [ ]:
# default_exp utils
from nbdev.showdoc import *
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Magnetic Laplacian (unfinished)
> Note: this currently attempts to create a *sparse* laplacian, but encounters difficulties working with complex numbers in sparse matrices.

The Magnetic Laplacian is an adaptation of normal Graph Laplacian to the setting of undirected graphs. Via a tunable parameter $q$, it allows manual specification of the level of undirected information one wishes to incorporate when training their models. Although the Magnetic Laplacian is only one among several means of extending the laplacian to directed graphs, it also appears "in nature", dating back to physics research in the 1990s.

The Magnetic Laplacian is defined through several steps, which we will walk through in turn. 

## Establishing a PyG dataset

Perlmutter et al. found the WebKB "Texas" and "Wisconsin" datasets amenable to directed graph pursuits. Each dataset describes the links between the web pages of a university, including student, class, faculty, and project webpages. As it is available through PyG, using it should (one hopes) be easy.

In [ ]:
from torch_geometric.datasets import WebKB

In [ ]:
dataset = WebKB('~/data',name = "Texas", transform=None, pre_transform=None)

There's only one graph in this dataset

So, let's extract it:

In [ ]:
data = dataset[0]

In [ ]:
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of nodes: 183
Number of edges: 325
Average node degree: 1.78
Number of training nodes: 870
Training node label rate: 4.75
Has isolated nodes: False
Has self-loops: True
Is undirected: False


The pyg graph objects have several standard features:
1. `.x` gives the node features, in rows
2. `.edge_index` gives the adjacency

In [ ]:
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
len(data.edge_index[0])

325

As seen above, the `edge_index` encodes graph connectivity in two long arrays, where a1[i] -> a2[i]. This is a sparse matrix format known as COO. Presently, we just care about the connectivity of the graph, so we'll separate this data into its own sparse tensor:

In [ ]:
import torch.sparse
A_directed = torch.sparse_coo_tensor(data.edge_index, torch.ones(data.num_edges),(data.num_nodes,data.num_nodes) )

Just to preview what this looks like:

In [ ]:
A_directed.to_dense()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

We form the *symmetrized adjacency matrix* as you'd expect, by averaging the two directions. Purely outgoing connections become 1/2 strength connections, while bidirectional links remain weighted as 1.

In [ ]:
A_symmetrized = 1/2*(A_directed + torch.t(A_directed))

In [ ]:
from torch import sparse
Degree = sparse.sum(A_symmetrized, dim = [1]).to_dense()

In [ ]:
Degree

tensor([ 1.0000,  0.5000,  0.5000,  0.5000,  1.5000,  1.5000,  1.0000,  1.0000,
         1.0000,  1.0000,  0.5000,  2.0000,  0.5000,  2.0000,  0.5000,  5.0000,
         4.5000,  1.5000,  1.5000,  0.5000,  2.0000,  1.5000,  2.0000,  2.5000,
         1.5000,  1.0000,  0.5000,  0.5000,  1.0000,  4.0000,  1.5000,  1.5000,
         0.5000,  0.5000,  5.0000,  0.5000,  1.0000,  1.0000,  0.5000,  2.0000,
         0.5000,  2.5000,  1.0000,  0.5000,  1.5000,  2.0000,  1.5000,  2.5000,
         1.0000,  1.0000,  1.5000,  0.5000,  0.5000,  1.0000,  1.5000,  2.0000,
        52.0000,  4.0000,  6.0000,  2.0000,  1.5000,  1.0000,  1.5000,  1.0000,
         2.0000,  1.5000,  6.5000,  1.5000,  1.0000,  0.5000,  0.5000,  0.5000,
         1.0000,  2.5000,  1.5000,  1.0000,  1.0000,  0.5000,  1.0000,  2.0000,
         2.5000,  1.0000,  4.5000,  2.0000, 10.0000,  1.5000,  1.5000,  1.0000,
         1.0000,  1.5000,  2.5000,  0.5000,  0.5000,  1.0000,  1.5000,  3.5000,
         0.5000,  1.0000,  0.5000,  1.50

And we define the *phase* matrix to capture the directional information so cruelly discarded by `A_symmetrized`.

In [ ]:
q = 0.25
Phase_matrix = 2*torch.pi*q*(A_directed - torch.t(A_directed))

## Diversion into Torch's Complex Numbers
Torch tensors have (beta) support for complex types, using the datatype `cfloat`. These can be instantiated with the function `torch.complex`, and, fortunately, the datatypes carry over intuitively: multiplication of real and complex values preserves the complex values.

In [ ]:
j = torch.complex(torch.zeros(1),torch.ones(1))


In [ ]:
j * torch.rand(3,3)

tensor([[0.+0.9586j, 0.+0.1400j, 0.+0.8984j],
        [0.+0.7221j, 0.+0.7698j, 0.+0.2241j],
        [0.+0.1167j, 0.+0.7521j, 0.+0.5159j]])

Unfortunately, Pytorch doesn't currently support scalar multiplication between a sparse matrix and a complex number. We have to do it the long way. Notably, we have to convert the Phase_matrix to have dtype of complex float, otherwise strange errors occur.

In [ ]:
Phase_matrix = Phase_matrix.cfloat()
imaginary_phase_matrix = torch.smm(Phase_matrix, torch.diag(torch.complex(torch.zeros(len(Phase_matrix)),torch.ones(len(Phase_matrix)))))

In [ ]:
Phase_matrix

tensor(indices=tensor([[  0,   0,   1,  ..., 180,  29, 182],
                       [ 58, 121,  80,  ..., 180, 182,  29]]),
       values=tensor([ 1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
                       1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,  1.5708+0.j,
      

Similarly, the exponential is not defined element-wise for sparse matrices, but we can do point-wise multiplication via the values.

In [ ]:
# Phase_matrix = Phase_matrix.coalesce()
# v_phase = imaginary_phase_matrix.coalesce().values()
# v_phase = torch.exp(v_phase)
# Phase_matrix_expd = torch.sparse_coo_tensor(Phase_matrix.indices(),v_phase)

In [ ]:
#len(v_phase)

In [ ]:
#torch.sum(v_phase[500:])

# Directed Diffusion Matrix
- [ ] Add support for edge weights

Util to take a directed adjacency matrix, and normalize it to a diffusion matrix. 

It accepts either sparse adjacency matrices, or PyG graphs, assuming torch's COO sparse format. Indeed, given a non-sparse adjacency matrix, it converts the matrix to a COO sparse matrix before continuing with the normalization

$$ P = D^{-1} A $$

The only difficulty lays in normalization: directed graphs may have rows with zero sum. To counteract that, we use a normalization function which takes the rowsums, and returns an indicator vector: 1 if zero, 0 otherwise. This is done in a slightly hacky way, by taking

$$ \text{floor}(2^{-rowsums}) $$

In [ ]:
# export
import torch
from torch import sparse
def diffusion_matrix_from_graph(A = None, G = None, self_loops=5):
  """
  Given directed adjacency matrix (sparse or unsparse), returns sparse diffusion matrix.
  Accepts tensor inputs of `A`, in COO sparse form, or dense, or can work directly from a PyG graph, given via argument `G`.
  """
  if G is not None:
    # We were given a graph. Extract the indices and values from it:
    A = torch.sparse_coo_tensor(G.edge_index, torch.ones(G.num_edges),(G.num_nodes,G.num_nodes) )
  if A is not None:
    # check if A is sparse
    if not A.is_sparse:
      A = A.to_sparse()
    if self_loops > 0:
      A = A + (self_loops * torch.eye(A.shape[0])).to_sparse()
    # We now have a sparse tensor: get row sums and set zeros equal to one
    # this prevents division by zero errors
    degree = sparse.sum(A, dim=[1]).to_dense()
    degree[degree == 0] = 1
    one_over_degree = 1 / degree
    D_negative_one = torch.diag(one_over_degree).to_sparse()
    # Row normalize by multiplying with a diagonal matrix
    P = D_negative_one @ A
    return P

We'll ensure this works with a few tests.

In [ ]:
# Does it work on matrix data?
X = torch.rand(20,20)
P = diffusion_matrix_from_graph(A = X)
# the maximum rowsum should be one, and the min should be either one or zero
max_row_sum = max(sparse.sum(P, dim=[1]).to_dense())
print(max_row_sum)
assert torch.allclose(max_row_sum, torch.ones(1))


tensor(1.0000)


In [ ]:
# Does it work on graph data?
P = diffusion_matrix_from_graph(G = data)
max_row_sum = max(sparse.sum(P, dim=[1]).to_dense())
print(max_row_sum)
assert torch.allclose(max_row_sum, torch.ones(1))

tensor(1.0000)


In [ ]:
torch.eye(10).to_sparse().shape

torch.Size([10, 10])

In [ ]:
!nbdev_build_lib

Converted 00_core.ipynb.
Converted 01_Diffusion Curvature of Directed Graphs.ipynb.
Converted 02_Directed_graph_utils.ipynb.
Converted 03_Toy_Graph_Datasets.ipynb.
Converted 12_differentiable_diffusion_curvature.ipynb.
Converted 21_Communities_Datasets.ipynb.
Converted index.ipynb.


In [ ]:
!nbdev_build_docs

converting: /Users/adjourner/Projects/directed_graphs/02_Directed_graph_utils.ipynb
converting /Users/adjourner/Projects/directed_graphs/index.ipynb to README.md
